<a href="https://colab.research.google.com/github/smichalka/premusings/blob/master/EEGNet_Pytorch_Sam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary

This code builds upon EEGNet (originally written using Keras here: https://github.com/vlawhern/arl-eegmodels/blob/master/EEGModels.py. The folder also includes links to some great papers and has excellent descriptions of their parameters). Here we are implementing in PyTorch. The starting parameters are actually derived from EEGNet_SSVEP. We will be applying this to our data, which uses an SSVEP paradigm.

Details of our data:
Participants attended to video with an overlayed flashing checkerboard at 12 or 15 Hz on the left side of a computer screen. The right side showed another video with a checkerboard flashing at the other frequency. Our goal here is to classify if a person was attending to the 12 or 15 Hz. 

This code was written by KF, Sam Michalka, and AD.


## Things to do next


*   Try downsampling the time data to something closer to 100 hz or 125 hz, potentially using average pooling across the time dimension (if this is appropriate)
* Fix the convolutions to make sure they are depthwise and pointwise, need to use groups, might need another conv
* Add in padding because right now your layers are shrinking in time dimension through convolution and you have to hard-code the sizes in. This might be preventable if you can get the temporal/bandpass filter to be an odd number, which is somewhat dependent on the sampling rate... so maybe 250 is good or 150 (but that's an uneven break into 500).
* add in plots to visualize the layers. See https://mc.ai/feature-visualisation-in-pytorch%E2%80%8A-%E2%80%8Asaliency-maps/ for ideas



* figure out how to set up cross validation, or determine this might not be appropriate (if the model takes too long to load), in which case you should still do a proper train/val split of the data. See https://skorch.readthedocs.io/en/stable/user/dataset.html



# Setup, imports, etc

## Setup Notes


*   Set hardware accelerator to GPU (runtime->change runtime type->change hardware accelerator to GPU), otherwise you may get an 'CUDA-capable device' error 

*   Will need to connect to Google Drive to import data (or load another way)



In [0]:
import csv
import numpy as np
import matplotlib.pyplot as plt
import time

import torch
import torch.nn as nn
import torch.optim as optim
#from torch.autograd import Variable
#import torch.nn.functional as F
#import torch.optim as optim
#from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score

!pip install torchviz
from torchviz import make_dot
from torch.autograd import Variable
from torch.utils import data
from torch.utils.data.sampler import SubsetRandomSampler

from torchvision import models
from torchsummary import summary #Allows you to look at the sizes of your layers

# Can skip these ones if just building with fake data
from google.colab import drive
drive.mount('/content/drive') # allow access to google drive as if it were a local filesystem
# Should probably just mount this folder not all of drive

# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
cudnn.benchmark = True

# Data format and loading
The data is shaped as a (1, Channels, Samples or Timepoints).


In [0]:
# Generate fake data to start
X_train = np.random.rand(32*4,1,32,1499).astype('float32') # np.random.rand generates between [0, 1)
y_train = np.round(np.random.rand(32*4).astype('float32')) # binary data, so we round it to 0 or 1.
# # y is conditions, so either 0 or 1

train_sampler = SubsetRandomSampler(
    np.arange(X_train.shape[0], dtype=np.int64))

# This part will be need to be repeated when you load real data 
srate = 500 #sampling rate in Hz
data_dims = X_train.shape[1:] #First dimension is number of trials/examples
print(data_dims)

Load data based on Katie's code

In [0]:
subj = "BN"

# Load the conditions
conditions = np.loadtxt(open("/content/drive/My Drive/HAL/Projects/EEGvideodataforMachineLearning/BN/train/conditions.csv", "rb"), delimiter=",", skiprows=1, dtype=int)
np.shape(conditions)[0]
y = conditions[:,1]%2  # Even conditions are 0 (high freq 15) and odd are 1 (low freq 12)
y= y.astype(np.float32)
print("Shape of y", y.shape)

# Load the raw data (X)
X = np.array([])
X = np.loadtxt(open("/content/drive/My Drive/HAL/Projects/EEGvideodataforMachineLearning/"+subj+"/train/1.csv", "rb"), delimiter=",", skiprows=1)
for i in range (1, y.shape[0]): # 531
    filepath = "/content/drive/My Drive/HAL/Projects/EEGvideodataforMachineLearning/"+subj+"/train/"+str(i+1)+".csv"
    # load data from filepath
    if (i-1)%100 == 0:
      print("Downloading data from:", filepath) 
    data = np.loadtxt(open(filepath, "rb"), delimiter=",", skiprows=1) 
    X = np.dstack((X, data))
print(np.shape(X))

# Rearrange to have number of trials first, then an empty dimension (channels for image, not ch for eeg)
X = np.moveaxis(X, -1,0)
print("Shape of X", np.shape(X))

X = np.expand_dims(X, axis=1)
print(np.shape(X)) # aiming for: (531, 1, 1499, 32)
X = X.astype(np.float32)

In [0]:
# Now make these Tensors and make them a dataset

In [0]:
# Divide up into train and val

tvboundary = 400

y_train = y[:tvboundary]
X_train = X[:tvboundary]
print(y_train.shape)

y_val = y[tvboundary:]
X_val = X[tvboundary:]
print(y_val.shape)

data_dims = X_train.shape[1:] #First dimension is number of trials/examples
print(data_dims)


#EEGNet_SSVEP Model

In [0]:
class EEGNet_SSVEP(nn.Module):
  """ Pytorch Implementation of EEGNet
    http://iopscience.iop.org/article/10.1088/1741-2552/aace8c/meta
    Note that this implements the newest version of EEGNet and NOT the earlier
    version (version v1 and v2 on arxiv). For example:
        
        1. Depthwise Convolutions to learn spatial filters within a 
        temporal convolution. The use of the depth_multiplier option maps 
        exactly to the number of spatial filters learned within a temporal
        filter. This matches the setup of algorithms like FBCSP which learn 
        spatial filters within each filter in a filter-bank. This also limits 
        the number of free parameters to fit when compared to a fully-connected
        convolution. 
        
        2. Separable Convolutions to learn how to optimally combine spatial
        filters across temporal bands. Separable Convolutions are Depthwise
        Convolutions followed by (1x1) Pointwise Convolutions. 
        
    
    While the original paper used Dropout, we found that SpatialDropout2D 
    sometimes produced slightly better results for classification of ERP 
    signals. However, SpatialDropout2D significantly reduced performance 
    on the Oscillatory dataset (SMR, BCI-IV Dataset 2A). We recommend using
    the default Dropout in most cases.
        
    Assumes the input signal is sampled at 128Hz. If you want to use this model
    for any other sampling rate you will need to modify the lengths of temporal
    kernels and average pooling size in blocks 1 and 2 as needed (double the 
    kernel lengths for double the sampling rate, etc). Note that we haven't 
    tested the model performance with this rule so this may not work well. 
    
    The model with default parameters gives the EEGNet-8,2 model as discussed
    in the paper. This model should do pretty well in general, although it is
	advised to do some model searching to get optimal performance on your
	particular dataset.
    We set F2 = F1 * D (number of input filters = number of output filters) for
    the SeparableConv2D layer. We haven't extensively tested other values of this
    parameter (say, F2 < F1 * D for compressed learning, and F2 > F1 * D for
    overcomplete). We believe the main parameters to focus on are F1 and D. 
    Inputs:
        
      nb_classes      : int, number of classes to classify
      Chans, Samples  : number of channels and time points in the EEG data
      dropoutRate     : dropout fraction
      kernLength      : length of temporal convolution in first layer. We found
                        that setting this to be half the sampling rate worked
                        well in practice. For the SMR dataset in particular
                        since the data was high-passed at 4Hz we used a kernel
                        length of 32.     
      F1, F2          : number of temporal filters (F1) and number of pointwise
                        filters (F2) to learn. Default: F1 = 8, F2 = F1 * D. 
      D               : number of spatial filters to learn within each temporal
                        convolution. Default: D = 2
      dropoutType     : Either SpatialDropout2D or Dropout, passed as a string.
      
      [1]. Waytowich, N. et. al. (2018). Compact Convolutional Neural Networks
    for Classification of Asynchronous Steady-State Visual Evoked Potentials.
    Journal of Neural Engineering vol. 15(6). 
    http://iopscience.iop.org/article/10.1088/1741-2552/aae5d8
    """
  def __init__(self):
    super(EEGNet_SSVEP,self).__init__()

    # Settings (can we potentially make this a function with input params?)
    nb_classes = 2
    Chans = data_dims[1]
    Samples = data_dims[2]
    kernLength = round(srate/2) #should be 250 for 500 Hz  #Actually want this to be an odd number to avoid padding issues, so may need to revisit or add padding manually
    print("Kernel length: ", kernLength)
    kernSepLength = 16
    F1 = 24
    F2 = 96  # must be divisible by F1*D?
    D = 2

    Favgpool1 = 4
    Favgpool2 = 8
    dropout_rate = .5  # might need to explore making this in one direction only?
    fc1_size = 10 # number of classes right now, might want to have another layer

    # BLOCK 1


    #self.avgpool1 = nn.AvgPool2d((1,Favgpool1))
    # Convolution in time (essentially band pass)
    self.convtime = nn.Conv2d(data_dims[0],F1,(1,kernLength),padding=0,padding_mode='replicate',bias=False)
    
    # Batch normalization (was axis=1 in keras, not sure equiv) 
    self.batchnorm1 = nn.BatchNorm2d(F1, eps=1e-05)
    
    # Depthwise convolution 
    self.convdepth = nn.Conv2d(F1,F1*D,(Chans,1),padding=0,padding_mode='replicate',bias=False,groups = F1) 
    # See https://pytorch.org/docs/master/generated/torch.nn.Conv2d.html#torch.nn.Conv2d for making this depthwise
    #If groups = nInputPlane, then it is Depthwise. 
    #If groups = nInputPlane, kernel=(K, 1), (and before is a Conv2d layer with groups=1 and kernel=(1, K)), then it is separable.
    
    # Batch normalization (was axis=1 in keras, not sure equiv) 
    self.batchnorm2 = nn.BatchNorm2d(F1*D, eps=1e-05)
    
    # Activation function internally in the network
    self.activation_func = torch.nn.ReLU()

    # Average pooling along the time dimension (downsampling)
    self.avgpool1 = nn.AvgPool2d((1,Favgpool1))
    # Followed by dropout

    # BLOCK 2
    self.convsep = nn.Conv2d(F1*D,F2,(1,kernSepLength), padding=0,padding_mode='replicate',bias=False, groups=F1*D)  #need bias and padding info
    # might need a second convolution here

    # Batch normalization (was axis=1 in keras, not sure equiv) 
    self.batchnorm3 = nn.BatchNorm2d(F2, eps=1e-05)
    
    # Need another round of conv, norm, activation, average  pooling
    self.avgpool2 = nn.AvgPool2d((1,Favgpool2))

    # Then flatten to linear

    # Fully connected layer - note the change in inputs 
    #self.output_size_2 = int(Chans*F2)
    self.fc1 = nn.Linear(37*F2, fc1_size)
    
    # Final fc layer to output. This is binary right now, but could also be number of classes, then you need to change the labels and loss
    self.fc2 = nn.Linear(fc1_size,1)

    # not following the EEGNET here, just trying to get a simple one
    #self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

    # maxpool_output_size is the total amount of data coming out of that
    # layer.  Explain why the line of code below computes this quantity.
    #self.maxpool_output_size = int(fc1_size * (data_dims[1] / 2) * (data_dims[2] / 2)) #div 2 b/c of either kern or stride

    # Add on a fully connected layer (like in our MLP)
    # fc stands for fully connected
    #self.fc1 = nn.Linear(self.maxpool_output_size, fc1_size)

    

    # Convert our fully connected layer into outputs that we can compare to the result
    #self.fc2 = nn.Linear(fc1_size, nb_classes)

  # The forward function in the class defines the operations performed on a given input to the model
  # and returns the output of the model
  def forward(self, x):
    # BLOCK 1
    x = self.convtime(x)
    x = self.batchnorm1(x)
    #x = self.pool(x)
    x = self.convdepth(x)
    x = self.batchnorm2(x)
    x = self.activation_func(x)
    x = self.avgpool1(x)
    x = nn.functional.dropout2d(x, .5) # Figure out how to put variables in here later
    #BLOCK 2
    x = self.convsep(x)
    #might need another conv step here to get pointwise right, also dims are shrinking so need to fix padding
    x = self.batchnorm3(x)
    x = self.activation_func(x)
    x = self.avgpool2(x)
    x = nn.functional.dropout2d(x, .5) # Figure out how to put variables in here later

    # Missing pointwise or separable convolution and several steps here, but just trying to get to linear while learning

    x = x.view(-1,37*96) #figure out the other dimension based on batch_size and the size of x; this should flatten the matrix
    x = self.fc1(x)
    #x = torch.sigmoid(x)
    #x = self.activation_func(x)
    x = self.fc2(x)
    x = torch.sigmoid(x)
    #x = torch.nn.Softmax(x) # This might only work if you go to the two classes, since it needs numbers to sum to 1
    return x

  # The loss function (which we chose to include as a method of the class, but doesn't need to be)
  # returns the loss and optimizer used by the model
  def get_loss(self, learning_rate):
    # Loss function
    #loss = nn.CrossEntropyLoss()
    # or you can use binary cross entropy
    loss = torch.nn.BCELoss()
    # Optimizer, self.parameters() returns all the Pytorch operations that are attributes of the class
    optimizer = optim.Adam(self.parameters(), lr=learning_rate)
    return loss, optimizer


def visualize_network(net):
    # Visualize the architecture of the model
    # We need to give the net a fake input for this library to visualize the architecture
    
    fake_input = Variable(torch.zeros((10,data_dims[0], data_dims[1], data_dims[2]))).to(device)
    outputs = net(fake_input)
    # Plot the DAG (Directed Acyclic Graph) of the model
    return make_dot(outputs, dict(net.named_parameters()))


# Initialize the model, loss, and optimization function
net = EEGNet_SSVEP()
# This tells our model to send all of the tensors and operations to the GPU (or keep them at the CPU if we're not using GPU)
net.to(device)

visualize_network(net)
summary(net,data_dims)

def evaluate(model, X, Y, batch_size, params = ["acc"]):
    results = []
    
    predicted = []
    
    for i in range(int(len(X)/batch_size)): # not sure if this should be changed
        s = i*batch_size
        e = i*batch_size+batch_size
        
        inputs = Variable(torch.from_numpy(X[s:e]).cuda(0))
        pred = model(inputs)
        
        predicted.append(pred.data.cpu().numpy())
        
        
    inputs = Variable(torch.from_numpy(X).cuda(0))
    predicted = model(inputs)
    
    predicted = predicted.data.cpu().numpy()
    
    for param in params:
        if param == 'acc':
            results.append(accuracy_score(Y, np.round(predicted)))
        if param == "auc":
            results.append(roc_auc_score(Y, predicted))
        if param == "recall":
            results.append(recall_score(Y, np.round(predicted)))
        if param == "precision":
            results.append(precision_score(Y, np.round(predicted)))
        if param == "fmeasure":
            precision = precision_score(Y, np.round(predicted))
            recall = recall_score(Y, np.round(predicted))
            results.append(2*precision*recall/ (precision+recall))
    return results

In [0]:
# Run the code to train and make the plots

# Define training parameters
batch_size = 25
learning_rate = 1e-3
n_epochs = 50


# Get our data into the mini batch size that we defined
#train_loader = torch.utils.data.DataLoader(X_train, batch_size=batch_size,
#                                           sampler=train_sampler, num_workers=2)
#print(train_loader)
#test_loader = torch.utils.data.DataLoader(
#    test_set, batch_size=128, sampler=test_sampler, num_workers=2)

def train_model(net):
    """ Train a the specified network.

        Outputs a tuple with the following four elements
        train_hist_x: the x-values (batch number) that the training set was 
            evaluated on.
        train_loss_hist: the loss values for the training set corresponding to
            the batch numbers returned in train_hist_x
        val_hist_x: the x-values (batch number) that the test set was 
            evaluated on.
        val_loss_hist: the loss values for the test set corresponding to
            the batch numbers returned in test_hist_x
    """ 
    loss, optimizer = net.get_loss(learning_rate)
    # Define some parameters to keep track of metrics
    print_every = 5
    idx = 0
    train_hist_x = []
    train_loss_hist = []
    val_hist_x = []
    val_loss_hist = []

    training_start_time = time.time()
    # Loop for n_epochs
    print('begin')
    for epoch in range(n_epochs):
      running_loss = 0.0
      start_time = time.time()

      # Loop through in batches on the train data
      for i in range(int((len(X_train)/batch_size)-1)):
        s = i*batch_size
        e = i*batch_size+batch_size
        inputs = torch.from_numpy(X_train[s:e])
        labels = torch.from_numpy(np.array([y_train[s:e]]).T*1.0) #if need float possibly for other loss func
        inputs, labels = Variable(inputs.to(device)), Variable(labels.to(device))  # wrap them in Variable
        # In Pytorch, We need to always remember to set the optimizer gradients to 0 before we recompute the new gradients
        optimizer.zero_grad()
        # Forward pass
        outputs = net(inputs)
        # Compute the loss and find the loss with respect to each parameter of the model
        loss_size = loss(outputs, labels)
        loss_size.backward()
        # Change each parameter with respect to the recently computed loss.
        optimizer.step()
        # Update statistics
        running_loss += loss_size.data.item()
        
        # Print every Nth batch of an epoch
        if (i % print_every) == print_every-1:
            print("Epoch {}, Iteration {}\t train_loss: {:.2f} took: {:.2f}s".format(
                epoch + 1, i+1,running_loss / print_every, time.time() - start_time))
            # Reset running loss and time
            train_loss_hist.append(running_loss / print_every)
            train_hist_x.append(idx)
            running_loss = 0.0
            start_time = time.time()
        idx += 1
      # At the end of the epoch, do a pass on the test set
      total_val_loss = 0
      for vi in range(int((len(X_val)/(batch_size))-1)):
          svi = vi*batch_size
          evi = vi*batch_size+batch_size
          valinputs = torch.from_numpy(X_val[svi:evi])
          vallabels = torch.from_numpy(np.array([y_val[svi:evi]]).T*1.0) #if need float possibly for other loss func
          # wrap them in Variable
          valinputs, vallabels = Variable(valinputs.to(device)), Variable(vallabels.to(device))  
          # Forward pass
          val_outputs = net(valinputs)
          val_loss_size = loss(val_outputs, vallabels)
          total_val_loss += val_loss_size.data.item()
      val_loss_hist.append(total_val_loss / len(X_val))
      val_hist_x.append(idx)
      print("Validation loss = {:.2f}".format(
          total_val_loss / len(y_val)))
      
    print("Training finished, took {:.2f}s".format(
        time.time() - training_start_time))
    return train_hist_x, train_loss_hist, val_hist_x, val_loss_hist

# RUN the TRAINING
train_hist_x, train_loss_hist, val_hist_x, val_loss_hist = train_model(net)

plt.plot(train_hist_x,train_loss_hist)
plt.plot(val_hist_x,val_loss_hist)
plt.legend(['train loss', 'validation loss'])
plt.xlabel('Batch number')
plt.ylabel('Loss')
plt.show()

In [0]:
print(X_val.shape)

In [0]:
# Plots of the conv kernels learned by the model
plt.figure()
plt.subplots(6, 4)
for i in range(net.convtime.weight.shape[0]):
    plt.subplot(6, 4, i+1)
    kernel = net.convtime.weight[i].cpu().detach().numpy()
    im = kernel.mean(axis=0)
    plt.pcolor(im, cmap='gray')
plt.show()

plt.figure()
plt.subplots(14, 6)
for i in range(net.convdepth.weight.shape[0]):
    plt.subplot(14, 6, i+1)
    kernel = net.convdepth.weight[i].cpu().detach().numpy()
    im = kernel.mean(axis=0)
    plt.pcolor(im, cmap='gray')
plt.show()

plt.figure()
plt.subplots(14, 6)
for i in range(net.convsep.weight.shape[0]):
    plt.subplot(14, 6, i+1)
    kernel = net.convsep.weight[i].cpu().detach().numpy()
    im = kernel.mean(axis=0)
    plt.pcolor(im, cmap='gray')
plt.show()